In [ ]:
from twikit import Client
from account_info import USERNAME, EMAIL, PASSWORD
import pandas as pd
import json
import urllib.request
import os
import time
import requests



# Initialize client
client = Client('en-US')

client.login(
    auth_info_1=USERNAME ,
    auth_info_2=EMAIL,
    password=PASSWORD
)

# save cookies in order to pull data without getting banned
client.save_cookies('cookies.json')

In [ ]:
# loads the cookies that were saved
client.load_cookies('cookies.json')

In [ ]:
from twikit import TooManyRequests

def download_image(URL, save_as):
    '''Function uses image URL and saves the image onto the desired path and file type'''
    urllib.request.urlretrieve(URL,save_as)

def delete_jpg(file):
    '''This file takes in the file path and deletes it.'''
    if os.path.exists(file):
        os.remove(file)
    else:
        print('File not found')

# this saves the json file on the described directory 
def save_json(file, file_path):
    with open(file_path, "w") as f:
        json.dump(file, f,indent=4)


def retry_on_rate_limit_error(func, *args, **kwargs):
    max_retries = 5
    retries = 0
    while retries < max_retries:
        try:
            return func(*args, **kwargs)
        except TooManyRequests as e:
            print("Rate limit exceed trying again in 60 sec")
            time.sleep(60)
            retries += 1
    raise Exception("Max tries reached")


In [ ]:

# file path to excel sheet
file_path = r"C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\src\utils\codes.xlsx"

def keywords_to_dict(file_path =r"C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\src\utils\codes.xlsx" ):
    '''This function takes in the excel sheet that has keywords and its ids split up into columns.
    Then it creates two dictionaries, keyname and keyname_id. The values are numbered 0 to the 
    length of the columns. Once created they return these dictionaries.'''

    # reads in excel sheet into pandas data frame
    df = pd.read_excel(file_path)

    # this turns the street name and its code name into list 
    street_name_list = df['Street Name'].astype(str).values.tolist()
    street_name_code_list = df['Street Name Code'].astype(str).values.tolist()

    # this dictionary contains the keyname in the keys and keyname id in values
    keyname = {}
    keyname_id = {}
    for i in range(len(street_name_list)):
        keyname[i] = street_name_list[i]
        keyname_id[i] = street_name_code_list[i]
        
    return keyname, keyname_id

# print(json.dumps(keyname, indent =4))


In [ ]:
keyname, keyname_id = keywords_to_dict()

users = {}

# goes through all the key names and searches them
for search_key in keyname.values():
    user_keyword = retry_on_rate_limit_error(client.search_user,search_key, 5)
    for user in user_keyword:

        try: 
            image_directoary = r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\profile_images'
            image_file_name = 'user_profile_pic_' + str(user.id) + '.jpg'
            image_save_path = os.path.join(image_directoary, image_file_name)
            download_image(user.profile_image_url, image_save_path)

            keywords = []

            keyname, keyname_id = keywords_to_dict()

            # get the users tweets
            tweets = user.get_tweets('tweets')

            # gets users followers
            followers = user.get_followers(user.id)

            # gets users following
            following = user.get_following(user.id)

            # empty list for the followers
            followers_list = []
            following_list = []

            # go through the users tweets, description, name to find keywords
            for keys, keyname in keyname.items():
                if keyname in user.name:
                    keywords.append(keyname_id[keys])
                elif keyname in user.screen_name:
                    keywords.append(keyname_id[keys])
                elif keyname in user.description:
                    keywords.append(keyname_id[keys])
                for tweet in tweets:
                    if keyname in tweet.text:
                        keywords.append(keyname_id[keys])

                # go through the users followers description, tweet, and names to 
                # see if it finds any keywords. If it does it adds them to the follower_list
                for follower_user in followers:
                    if keyname in follower_user.name:
                        followers_list.append('user_' +  user.id)
                    elif keyname in follower_user.screen_name:
                        followers_list.append('user_' + user.id)
                    elif keyname in follower_user.description:
                        followers_list.append('user_' + user.id)
                    for tweet in tweets:
                        if keyname in tweet.text:
                            followers_list.append('user_' + user.id)

                # Goes thorugh the users following description, tweet, and names to 
                # see if it finds any keywords. If it does it adds them to the following_list
                for following_user in following:
                    if keyname in following_user.name:
                        following_list.append('user_' +  user.id)
                    elif keyname in following_user.screen_name:
                        following_list.append('user_' + user.id)
                    elif keyname in following_user.description:
                        following_list.append('user_' + user.id)
                    for tweet in tweets:
                        if keyname in tweet.text:
                            following_list.append('user_' + user.id)

            # Structure to store the data
            User_Structure = {
            "username": user.screen_name,
            "user_id": user.id, 
            "followers": [follower for follower in followers_list], #['user_' + str(follower_id) for follower_id in client.get_followers_ids(user.id,user.screen_name,30)],
            "followees": [following for following in following_list], #['user_' + str(followees_id) for followees_id in client.get_friends_ids(user.id,user.screen_name,30)],
            "profile_pic": image_save_path,
            "profile_text": user.description,
            "posts": ['tweet_' + tweet.id for tweet in tweets]  ,
            "keywords": keywords
            }

            # save the user to the dictionary
            name = 'user_'+ user.id
            if name not in users:
                users[name] = User_Structure
        except TooManyRequests as e:
            print('Rate limit exceeced')
            break

save_path = r'C:\Users\16822\Research Project SER\SEP-NHANES\lmolina3\data\data_for_twitter\user_structure.json'
save_json(users,save_path)
print(json.dumps(users, indent =4))

In [ ]:
print(users)